<a href="https://colab.research.google.com/github/lightyisu/colab_collection/blob/main/dna%E5%88%86%E5%89%B2%E5%AE%9E%E9%AA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
from transformers import AutoModel

from memory_profiler import profile
class BERTModel(nn.Module):
    def __init__(self, bert_path, label_count):
        super(BERTModel, self).__init__()
        self.bert = AutoModel.from_pretrained('/kaggle/working/DNABERT-2-117M',trust_remote_code=True)
        self.num_labels = label_count
        self.dropout = nn.Dropout(0.1)
        self.loss_func = nn.CrossEntropyLoss()
        self.linear = nn.Linear(768, label_count)
    def forward(self, input_ids=None, label_ids=None, mask=None):
        print('oxp')
        print(input_ids.shape)
        input_temp=input_ids
        outputs = self.bert(input_ids=input_ids)
        print('op',outputs[0])
        # sequence_output = outputs[0]
        # sequence_output = self.dropout(sequence_output)
        # logits = self.linear(sequence_output)
        # outputs = (logits,) + outputs[2:]
        # active_loss = mask.view(-1) == 1

        # if label_ids is not None:
        #     if mask is not None:
        #         active_loss = mask.view(-1) == 1
        #         active_logits = logits.view(-1, self.num_labels)[active_loss]
        #         active_labels = label_ids.view(-1)[active_loss]
        #         loss = self.loss_func(active_logits, active_labels)
        #     outputs = (loss,) + outputs

        return outputs

class BERTTest(nn.Module):
    def __init__(self, bert_path, label_count):
        super(BERTTest, self).__init__()
        self.num_labels = label_count
        self.bert =AutoModel.from_pretrained(bert_path,trust_remote_code=True)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(768, label_count)
    def forward(self, input_ids=None, mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.linear(sequence_output)
        if mask is not None:
            active_mask = mask.view(-1) == 1
            active_logits = logits.view(-1, self.num_labels)[active_mask]
        return  active_logits

In [ ]:
input_temp=None

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset,DataLoader
label2id = {
    "0": 0,
    "1": 1

}




class DataProcess():
    def __init__(self, data_path, data_type):
        self.data_dir = os.path.join(data_path, data_type+'.npz')
    def process(self):
        data = np.load(self.data_dir, allow_pickle=True)
        data_df = pd.concat([pd.DataFrame(data['words'], columns=['words']),
                            pd.DataFrame(data['labels'], columns=['labels'])],axis=1)
        data_df = data_df.dropna()
        # data_df['labels'] = data_df['labels'].map(lambda x: self.trans(x))
        corpus = []
        for _, row in data_df.iterrows():
            words = row['words']
            labels = row['labels']
            corpus.append((words, labels))
        return corpus

    # def trans(self, labels):
    #     labels = list(labels)
    #     nums = [label2id[label] for label in labels]
    #     return nums



class CluenerDataset(Dataset):
    def __init__(self, corpus, tokenizer=None, seq_len=50):
        super(CluenerDataset, self).__init__()
        self.corpus = corpus
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.len = len(corpus)



    def __getitem__(self, item):
        sentence, label_ids = self.corpus[item]

        # tokens = ['[CLS]'] + tokens + ['[SEP]']
        # label_ids = [0] + label_ids + [0]

        max_length = 312
        input_ids = self.tokenizer(sentence, return_tensors='pt',padding='max_length', truncation=True,max_length=max_length)["input_ids"]
#         print('batch',input_ids)

        label_ids=[int(label) for label in label_ids]

        return {"input_ids": input_ids ,
                        "label_ids": torch.IntTensor(label_ids) }

    def __len__(self):
        return self.len



def build_loader(data_path, data_type,tokenizer=None,seq_len=50):
    dataprocess = DataProcess(data_path, data_type)
    corpus = dataprocess.process()
    dataset = CluenerDataset(corpus,tokenizer,seq_len)
    data_loader = DataLoader(dataset, batch_size=8,shuffle=False)
    return data_loader

In [ ]:

import random
import numpy as np
import torch
from tqdm import tqdm
from pathlib import Path
from tqdm import trange
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from transformers import AutoTokenizer

from torch.utils.tensorboard import SummaryWriter

import torch, gc



def train_model(model, epochs, train_loader,save_path,log_dir, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    for epoch in trange(epochs):
        tr_loss, n_steps, correct_preds, total_preds = 0, 0, 0, 0
        model.train()
        writer = SummaryWriter(log_dir=log_dir + f'/run_{epoch}')
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
        for _, batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            label_ids = batch['label_ids'].to(device)
            # mask = batch['attn_mask'].to(device)
            print('start-----')
            output = model(input_ids, label_ids )
            print('end-----')
            break;
        epoch_loss = tr_loss / n_steps
        epoch_acc = correct_preds / total_preds
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
        writer.add_scalar('Train/Loss', epoch_loss, epoch + 1)
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f'{save_path}/{epoch + 1}.pt')
    writer.close()

def main():
    bert_path = "/kaggle/working/DNABERT-2-117M"
    save_path = '/kaggle/working/dnaseq'
    log_dir = '/kaggle/working/dnaseq'
    input_path = "/kaggle/working"
    data_type = "train"
    tokenizer = AutoTokenizer.from_pretrained(bert_path,trust_remote_code=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader = build_loader(data_path= input_path,data_type=data_type,tokenizer=tokenizer, seq_len=310)
    model = BERTModel(bert_path=bert_path,label_count=2).to(device)
    train_model(model, 100, train_loader, save_path, log_dir, device)


main()

/root/.cache/huggingface/modules/transformers_modules/DNABERT-2-117M/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at /kaggle/working/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/100 [00:00<?, ?it/s] 

start-----
oxp
torch.Size([8, 300, 312])


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.14 GiB. GPU 0 has a total capacty of 14.75 GiB of which 1.31 GiB is free. Process 3256 has 13.44 GiB memory in use. Of the allocated memory 11.62 GiB is allocated by PyTorch, and 1.71 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
!wget https://pan.tenire.com/down.php/e6f91824c1a792446bd9d8afb45bcdee.npz -O train.npz

--2024-06-04 12:03:07--  https://pan.tenire.com/down.php/e6f91824c1a792446bd9d8afb45bcdee.npz
Resolving pan.tenire.com (pan.tenire.com)... 188.114.97.0, 188.114.96.0, 2a06:98c1:3121::, ...
Connecting to pan.tenire.com (pan.tenire.com)|188.114.97.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64784459 (62M) [application/force-download]
Saving to: 'train.npz'

train.npz           100%[===================>]  61.78M  39.4MB/s    in 1.6s    

2024-06-04 12:03:08 (39.4 MB/s) - 'train.npz' saved [64784459/64784459]



In [ ]:
!git lfs clone https://huggingface.co/zhihan1996/DNABERT-2-117M

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'DNABERT-2-117M'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 68 (delta 28), reused 5 (delta 5), pack-reused 22 (from 1)
Unpacking objects: 100% (68/68), 83.60 KiB | 2.79 MiB/s, done.


In [ ]:
!pip install einops

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 629.1 kB/s eta 0:00:00a 0:00:01


In [ ]:
!git lfs clone https://huggingface.co/zhihan1996/DNABERT-2-117M

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
fatal: destination path 'DNABERT-2-117M' already exists and is not an empty directory.
Error(s) during clone:
git clone failed: exit status 128


In [ ]:
!pip install einops

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/DNABERT-2-117M",trust_remote_code=True)
model = AutoModel.from_pretrained("/kaggle/working/DNABERT-2-117M",trust_remote_code=True)


Some weights of BertModel were not initialized from the model checkpoint at /kaggle/working/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
def _tokenize_extend_labels(sentence):
        tokens = []
        for word in sentence:
            tokenized_word = tokenizer.tokenize(word)
            tokens.extend(tokenized_word)
        return tokens
# str=['生', '生', '不', '息', 'C', 'S', 'O', 'L', '生', '化', '狂', '潮', '让', '你', '填', '弹', '狂', '扫']
str=['A', 'C', 'G', 'T', 'A', 'G', 'C', 'A', 'T', 'C', 'G', 'A', 'C', 'A', 'C', 'T', 'T', 'G', 'G', 'T', 'T', 'A', 'T', 'C', 'G', 'A', 'T', 'A', 'G', 'C']
# tokens=tokenizer.tokenize(str)

tokens=_tokenize_extend_labels(str)
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)
#directly tokenizer -shape([18,3]) 3-single tokenzer-id
#([[ 101, 4495,  102],
#[ 101, 4495,  102]])
inputs_id=tokenizer.convert_tokens_to_ids(tokens)
inputs_id=torch.tensor(inputs_id, dtype=torch.long)
inputs_id=inputs_id.unsqueeze(1)
# inputs_id=tokenizer(str)["input_ids"]
# inputs_id=tokenizer(str)["input_ids"]
hidden_states = model(inputs_id)[0] # [1, sequence_length, 768]
# print(inputs_id)
print('hidden_state',hidden_states)

['[CLS]', 'A', 'C', 'G', 'T', 'A', 'G', 'C', 'A', 'T', 'C', 'G', 'A', 'C', 'A', 'C', 'T', 'T', 'G', 'G', 'T', 'T', 'A', 'T', 'C', 'G', 'A', 'T', 'A', 'G', 'C', '[SEP]']
hidden_state tensor([[[ 0.0251, -0.1064, -0.0880,  ..., -0.0253,  0.0708,  0.0632]],

        [[ 0.0490,  0.0624,  0.2037,  ..., -0.0567,  0.1172, -0.0316]],

        [[ 0.1169,  0.0800,  0.5125,  ...,  0.1701,  0.3980,  0.2334]],

        ...,

        [[-0.0821, -0.1935,  0.0383,  ...,  0.0619,  0.1893,  0.0249]],

        [[ 0.1169,  0.0800,  0.5125,  ...,  0.1701,  0.3980,  0.2334]],

        [[-0.0585, -0.1229, -0.1667,  ..., -0.0962,  0.0483, -0.0757]]],
       grad_fn=<ViewBackward0>)


In [ ]:
str="ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
print([item for item in str])

['A', 'C', 'G', 'T', 'A', 'G', 'C', 'A', 'T', 'C', 'G', 'G', 'A', 'T', 'C', 'T', 'A', 'T', 'C', 'T', 'A', 'T', 'C', 'G', 'A', 'C', 'A', 'C', 'T', 'T', 'G', 'G', 'T', 'T', 'A', 'T', 'C', 'G', 'A', 'T', 'C', 'T', 'A', 'C', 'G', 'A', 'G', 'C', 'A', 'T', 'C', 'T', 'C', 'G', 'T', 'T', 'A', 'G', 'C']


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese",trust_remote_code=True)
model = AutoModel.from_pretrained("google-bert/bert-base-chinese",trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [ ]:
import torch
def _tokenize_extend_labels(sentence):
        tokens = []
        for word in sentence:
            tokenized_word = tokenizer.tokenize(word)
            tokens.extend(tokenized_word)
        return tokens
str=['生', '生', '不', '息', 'C', 'S', 'O', 'L', '生', '化', '狂', '潮', '让', '你', '填', '弹', '狂', '扫']
# str=['A', 'C', 'G', 'T', 'A', 'G', 'C', 'A', 'T', 'C', 'G', 'A', 'C', 'A', 'C', 'T', 'T', 'G', 'G', 'T', 'T', 'A', 'T', 'C', 'G', 'A', 'T', 'A', 'G', 'C']
# tokens=tokenizer.tokenize(str)

tokens=_tokenize_extend_labels(str)
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)
#directly tokenizer -shape([18,3]) 3-single tokenzer-id
#([[ 101, 4495,  102],
#[ 101, 4495,  102]])
inputs_id=tokenizer.convert_tokens_to_ids(tokens)
inputs_id=torch.tensor(inputs_id, dtype=torch.long)
inputs_id=inputs_id.unsqueeze(1)
# inputs_id=tokenizer(str)["input_ids"]
# inputs_id=tokenizer(str)["input_ids"]
hidden_states = model(inputs_id)[0] # [1, sequence_length, 768]
print(inputs_id)
print('hidden_state',hidden_states)

['[CLS]', '生', '生', '不', '息', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '生', '化', '狂', '潮', '让', '你', '填', '弹', '狂', '扫', '[SEP]']
tensor([[ 101],
        [4495],
        [4495],
        [ 679],
        [2622],
        [ 100],
        [ 100],
        [ 100],
        [ 100],
        [4495],
        [1265],
        [4312],
        [4060],
        [6375],
        [ 872],
        [1856],
        [2486],
        [4312],
        [2812],
        [ 102]])
hidden_state tensor([[[ 0.1157,  0.3340, -0.3357,  ...,  1.7525, -0.5618,  0.3029]],

        [[ 1.0068,  0.6529,  0.5228,  ...,  0.9169, -0.1935, -0.1904]],

        [[ 1.0068,  0.6529,  0.5228,  ...,  0.9169, -0.1935, -0.1904]],

        ...,

        [[ 0.6826,  1.0743, -0.4017,  ...,  1.3547, -0.3943,  0.0812]],

        [[ 0.1855, -0.4305,  0.0671,  ...,  0.9354,  0.4295, -0.1573]],

        [[ 0.1160,  0.3337, -0.3363,  ...,  1.7524, -0.5612,  0.3031]]],
       grad_fn=<NativeLayerNormBackward0>)
